# Zurich Instruments Hardware Setup

This notebook shows an exemplary use of qupulse with a ZI HDAWG and MFLI. The drivers for these instruments are kept in external packages to facilitate easy driver customization. Depending on your python version and hardware version you either need `qupulse-hdawg-legacy` or `qupulse-hdawg` for the HDAWG and `qupulse-mfli` for the MFLI.

## Connections and wiring

The example here assumes a very nonsensical wiring that does not require anything else besides an HDAWG, and MFLI and three cables/adapters to connect SMB to BNC ports.

## Hardware Setup

The hardware setup class provides a layer to map output channels to an arbitrary number of physical channels.
It also provides a mapping of measurement windows to specific dac instruments

In [ ]:
from qupulse.hardware.setup import HardwareSetup

hw_setup = HardwareSetup()

In [ ]:
# This abstracts over possibly installed hdawg drivers
from qupulse.hardware.awgs.zihdawg import HDAWGRepresentation

awg_serial = 'DEVXXXX'
assert awg_serial != 'DEVXXXX', "Please enter the serial of a connected HDAWG"

hdawg = HDAWGRepresentation(awg_serial)

### Channel groupings

The `AWG` class abstracts over a set of dependently programmable channels. The HDAWG supports multiple channel groupings which decouples individual channel groups. The most robust setting for qupulse is to use the `1x8` channel grouping which executes the same sequencing program on all channels and only differs in the waveform data that is sequenced. This results in a single channel tuple/`AWG` object which represents all eight channels.



In [ ]:
from qupulse.hardware.awgs.zihdawg import HDAWGChannelGrouping
from qupulse.hardware.setup import PlaybackChannel, MarkerChannel

hdawg.channel_grouping = HDAWGChannelGrouping.CHAN_GROUP_1x8
awg, = hdawg.channel_tuples

# here we assume plunger one and two are connected to the two first channels of the AWG
# It is considered best practice to use such names that relate to the connected sample gates
hw_setup.set_channel('P1', PlaybackChannel(awg, 0))
hw_setup.set_channel('P2', PlaybackChannel(awg, 1))

# We connect the trigger to the marker output of the first channel
hw_setup.set_channel('Trig', MarkerChannel(awg, 0))

# We can assign the same channel to multiple identifiers. Here we just assign all channels to a hardware name
for channel_idx, channel_letter in enumerate('ABCDEFGH'):
    channel_name = f"{hdawg.serial}_{channel_letter}"
    hw_setup.set_channel(channel_name, PlaybackChannel(awg, channel_idx), allow_multiple_registration=True)

# We can also assign multiple channels to the same identifier
hw_setup.set_channel(f"{hdawg.serial}_ALL", [PlaybackChannel(awg, idx) for idx in range(8)])

## MFLI

Next we will connect the MFLI.

In [ ]:
from qupulse_mfli.mfli import MFLIDAQ

mfli_serial = 'DEVXXXX'
assert mfli_serial != 'DEVXXXX', "Please enter the serial of a connected MFLI"

mfli = MFLIDAQ.connect_to(mfli_serial)

So

In [ ]:
from qupulse.hardware.setup import MeasurementMask

hw_setup.set_measurement('SET1', MeasurementMask(mfli, 'AverageR'))
hw_setup.set_measurement('SET2', MeasurementMask(mfli, 'AverageAux1'))

raise NotImplementedError("TODO: Implement the rest of the MFLI operations and so on")

from

We define the pulse template in terms of the potentials of quantum dot one and two `Q1` and `Q2` and provide a linear transformation that maps them to the output voltages `P1` and `P2`.

In [ ]:
from qupulse.pulses import *
import numpy as np
from qupulse.program.transformation import LinearTransformation

pt = (ConstantPT(2**20, {
    'Q1': '-0.1 + x_i * 0.02',
    'Q2': '-0.2 + y_i * 0.05'})
      .with_iteration('x_i', 'N_x')
      .with_iteration('y_i', 'N_y'))

trafo = LinearTransformation(np.array([[1., -.1], [-.09, 1.]]),
                             ('Q1', 'Q2'),
                             ('P1', 'P2'))

program = pt.create_program(parameters={'N_x': 50, 'N_y': 30}, global_transformation=trafo)

In [ ]:
hw_setup.register_program('csd', program, awg.run_current_program)

In [ ]:
hw_setup.arm_program('csd')

In [ ]:
hw_setup.run_program('csd')

The data extration is not standardized at the time of writing this example because it heavily depends on your data processing pipeline how the data is handled and where it shall go. qupulse has no functionality to associate a measured value with the value of some parameter that might have been varied during the measurement.